Since we are moving toward a more build your own analyses approach, I figured this cross correlation analysis would be a good place to start. I will load modules as needed (as opposed to at the very beginning).

# Part 1: Preparing Velocity Data

To start, we need to load and prepare the velocity data. Since it is in an ethovision format, we will borrow some of the code from `behavioral_analysis`. This part of the analysis will take place in three stages:

1. Load velocity data from the ethovision file
2. Get 